In [1]:
import sqlite3
import pandas as pd
import tempun 
import geopandas as gpd
import re

In [2]:
conn = sqlite3.connect('large_files/cultura_12_2024.db')

# Chose a table to extract
table_name = 'individuals_main_information'

# Load the table as a pandas DataFrame
cultura = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

In [3]:
cultura.shape

(220770, 8)

In [4]:
cultura.head(10)

,individual_wikidata_id,individual_name,birthyear,individual_impact_years,country_code,country_name,wikipedia_cultural_score,identifier_score
0,Q55844134,Ernst von Heeringen,1810.0,1820-1870,None,None,NaN,6.0
1,Q202809,Frederick Coombs,1803.0,1810-1860,GBR,United Kingdom,0.102597,12.0
2,Q94835418,Georg Kurtscholz,1847.0,1860-1910,None,None,NaN,4.0
3,Q33934701,Margarethe Sömmering,1768.0,1780-1830,None,None,0.100135,1.0
4,Q4350107,Vassili Perevoštšikov,1785.0,1800-1850,RUS,Russia,0.101319,4.0
5,Q1028749,Christian Friedrich Zincke,1683.0,1690-1740,GBR,United Kingdom,0.100409,37.0
6,Q2822993,Achard of Saint-Victor,1100.0,1110-1160,GBR,United Kingdom,0.101588,46.0
7,Q105607234,Angelo Dalbon,1787.0,1800-1850,ITA,Italy,0.100120,4.0
8,Q91771,Hans Krumpper,1570.0,1580-1630,DEU,Germany,0.101971,24.0
9,Q5362982,Elizabeth Hoby,1528.0,1540-1590,GBR,United Kingdom,0.101881,23.0


In [5]:
cultura = cultura[cultura["individual_impact_years"].notnull()]

In [6]:
years = "-20-30"
re.search("(\-?\d+)(\-)(\-?\d+)", years).groups()

('-20', '-', '30')

In [7]:
cultura["not_before"] = cultura["individual_impact_years"].apply(
    lambda x: int(re.search("(\-?\d+)(\-)(\-?\d+)", years).groups()[0]))
cultura["not_after"] = cultura["individual_impact_years"].apply(
    lambda x: int(re.search("(\-?\d+)(\-)(\-?\d+)", years).groups()[2]))

In [8]:
cultura["random_dates"] = cultura.apply(lambda row: tempun.model_date(row["not_before"], row["not_after"], size=100), axis=1)

In [11]:
((cultura["birthyear"] <= 600) & (cultura["country_code"]=="GRC")).sum()

545

In [12]:
((cultura["birthyear"] <= 600) & (cultura["country_code"]=="ITA")).sum()

454

In [13]:
cultura_ant_grc = cultura[(cultura["birthyear"] <= 600) & (cultura["country_code"]=="GRC")]
cultura_ant_ita = cultura[(cultura["birthyear"] <= 600) & (cultura["country_code"]=="ITA")]

In [15]:
# download and load the epigraphic data
try:
    LIST = gpd.read_parquet("large_files/LIST_v1-2.parquet")
    GIST = gpd.read_parquet("large_files/GIST_v1-1.parquet")
except:
    !curl https://zenodo.org/records/10473706/files/LIST_v1-2.parquet -o large_files/LIST_v1-2.parquet
    !curl https://zenodo.org/records/10139110/files/GIST_v1-1.parquet -o large_files/GIST_v1-1.parquet
    LIST = gpd.read_parquet("large_files/LIST_v1-2.parquet")
    GIST = gpd.read_parquet("large_files/GIST_v1-1.parquet")

In [ ]:
LIST["random_dates"] = LIST.apply(lambda row: tempun.model_date(row["not_before"], row["not_after"], size=100), axis=1)
GIST["random_dates"] = GIST.apply(lambda row: tempun.model_date(row["not_before"], row["not_after"], size=100), axis=1)

In [ ]:
tempun.kdeplot_from_randoms(cultura_ant_grc["random_dates"])